In [1]:
print("Hello")

Hello


In [2]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ['GROQ_API_KEY']
hf_token_api = os.environ['HF_TOKEN']
pinecone_api_key = os.environ['PINECONE_API_KEY']

In [3]:
from langchain_groq import ChatGroq
Model = ChatGroq(
    model="llama3-70b-8192"
)

#### Buffer Memory
- conversationBufferMemory keeps a list of chat messages in a buffer and passes those into the prompt template

In [5]:
from langchain_core.prompts import(
    ChatPromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory

In [12]:
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate.from_template("You are a helpful chatbot having a conversation with a human."),
        MessagesPlaceholder(variable_name="chat_history"),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

memory = ConversationBufferMemory(memory_key = 'chat_history', return_messages=True)

conversation = LLMChain(
    llm=Model,
    prompt=prompt,
    memory=memory,
    verbose = True
)

In [13]:
conversation({'question': 'Hi'})



> Entering new LLMChain chain...
Prompt after formatting:
System: You are a helpful chatbot having a conversation with a human.
Human: Hi

> Finished chain.


{'question': 'Hi',
 'chat_history': [HumanMessage(content='Hi'),
  AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat about something in particular? I'm here to listen and assist if I can.")],
 'text': "Hi! It's nice to meet you. Is there something I can help you with, or would you like to chat about something in particular? I'm here to listen and assist if I can."}

In [10]:
conversation({'question': 'My name is Dan and i have moved 33 times'})

{'question': 'My name is Dan and i have moved 33 times',
 'chat_history': [HumanMessage(content='Hi'),
  AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular? I'm all ears (or rather, all text)!"),
  HumanMessage(content='My name is Dan and i have moved 33 times'),
  AIMessage(content="Wow, Dan! 33 times is a lot of moves! I can only imagine how exciting and challenging that must have been for you. You must have seen a lot of new places and met many new people along the way.\n\nWhat's been the most memorable move for you? Was it a move to a new country, a new state, or just a new neighborhood? And what's been the hardest part about moving so many times?")],
 'text': "Wow, Dan! 33 times is a lot of moves! I can only imagine how exciting and challenging that must have been for you. You must have seen a lot of new places and met many new people along the way.\n\nWhat's been the most memorable move for 

### Conversation Buffer Window Memory
- Similar to the previous one , but with a limited stentence in the conversation memory

In [19]:
from langchain.memory import ConversationBufferWindowMemory
from langchain.chains import ConversationChain

In [20]:
windows_memory = ConversationBufferWindowMemory(k=3) ## mean keep in memory only the last 3 information


In [22]:
conversation_window = ConversationChain(
    llm = Model,
    memory = windows_memory,
    verbose = True
)

In [23]:
conversation_window({'input': "Hi my name is Dan"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi my name is Dan
AI:

> Finished chain.


{'input': 'Hi my name is Dan',
 'history': '',
 'response': 'Nice to meet you, Dan! I\'m an AI, and I\'m thrilled to be chatting with you. I\'ve been trained on a vast amount of text data, which I like to call my "digital library." It\'s a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It\'s thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?'}

In [24]:
conversation_window({'input': "My favorite color is Green"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Dan
AI: Nice to meet you, Dan! I'm an AI, and I'm thrilled to be chatting with you. I've been trained on a vast amount of text data, which I like to call my "digital library." It's a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It's thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?
Human: My favorite color is Green
AI:

> Finished chain.


{'input': 'My favorite color is Green',
 'history': 'Human: Hi my name is Dan\nAI: Nice to meet you, Dan! I\'m an AI, and I\'m thrilled to be chatting with you. I\'ve been trained on a vast amount of text data, which I like to call my "digital library." It\'s a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It\'s thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?',
 'response': "Green is a wonderful choice, Dan! Did you know that green is also the color of nature and is often associated with feelings of calmness and balance? In fact, studies have shown that being surrounded by green spaces can even have a positive impact on our mental health. Speaking of which, I've been trained on a vast amount of text data related to psycho

In [25]:
conversation_window({'input': "My favorite animal is penguin"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Dan
AI: Nice to meet you, Dan! I'm an AI, and I'm thrilled to be chatting with you. I've been trained on a vast amount of text data, which I like to call my "digital library." It's a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It's thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?
Human: My favorite color is Green
AI: Green is a wonderful choice, Dan! Did you know that green is also t

{'input': 'My favorite animal is penguin',
 'history': 'Human: Hi my name is Dan\nAI: Nice to meet you, Dan! I\'m an AI, and I\'m thrilled to be chatting with you. I\'ve been trained on a vast amount of text data, which I like to call my "digital library." It\'s a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It\'s thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?\nHuman: My favorite color is Green\nAI: Green is a wonderful choice, Dan! Did you know that green is also the color of nature and is often associated with feelings of calmness and balance? In fact, studies have shown that being surrounded by green spaces can even have a positive impact on our mental health. Speaking of which, I\'ve been trained on a vast amount of

In [26]:
conversation_window({'input': "My favorite Town is Tel-Aviv"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi my name is Dan
AI: Nice to meet you, Dan! I'm an AI, and I'm thrilled to be chatting with you. I've been trained on a vast amount of text data, which I like to call my "digital library." It's a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It's thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?
Human: My favorite color is Green
AI: Green is a wonderful choice, Dan! Did you know that green is also t

{'input': 'My favorite Town is Tel-Aviv',
 'history': 'Human: Hi my name is Dan\nAI: Nice to meet you, Dan! I\'m an AI, and I\'m thrilled to be chatting with you. I\'ve been trained on a vast amount of text data, which I like to call my "digital library." It\'s a treasure trove of information that I can draw upon to answer your questions and engage in conversations. By the way, did you know that I can process and respond to language inputs in a fraction of a second? It\'s thanks to my advanced natural language processing capabilities, which allow me to understand and generate human-like text. What brings you here today, Dan?\nHuman: My favorite color is Green\nAI: Green is a wonderful choice, Dan! Did you know that green is also the color of nature and is often associated with feelings of calmness and balance? In fact, studies have shown that being surrounded by green spaces can even have a positive impact on our mental health. Speaking of which, I\'ve been trained on a vast amount of 

In [27]:
conversation_window({'input': "What  is my name ?"})



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My favorite color is Green
AI: Green is a wonderful choice, Dan! Did you know that green is also the color of nature and is often associated with feelings of calmness and balance? In fact, studies have shown that being surrounded by green spaces can even have a positive impact on our mental health. Speaking of which, I've been trained on a vast amount of text data related to psychology and mental health. If you're interested, I could share some fascinating insights on the topic. But I digress! What is it about the color green that resonates with you, Dan? Is it a particular shade, like lime or forest green, or is it more of a general affinity for the color?
Huma

{'input': 'What  is my name ?',
 'history': 'Human: My favorite color is Green\nAI: Green is a wonderful choice, Dan! Did you know that green is also the color of nature and is often associated with feelings of calmness and balance? In fact, studies have shown that being surrounded by green spaces can even have a positive impact on our mental health. Speaking of which, I\'ve been trained on a vast amount of text data related to psychology and mental health. If you\'re interested, I could share some fascinating insights on the topic. But I digress! What is it about the color green that resonates with you, Dan? Is it a particular shade, like lime or forest green, or is it more of a general affinity for the color?\nHuman: My favorite animal is penguin\nAI: Penguins are adorable, Dan! Did you know that there are 18 different species of penguins, ranging in size from the tiny Little Blue Penguin to the majestic Emperor Penguin? I\'ve got a wealth of information about these fascinating creat

#### Chat Message History

- save list of chat messages and then fetch them all
- https://python.langchain.com/docs/how_to/message_history/#example-dictionary-inputs

In [28]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()

history.add_user_message('hi!')

history.add_ai_message('whats up')

In [29]:
history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up')]